In [1]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from numpy.random import randint, choice, seed

In [2]:
corpus = pd.read_csv('sentence_tokens.tsv', sep = '\\t', engine = 'python')

In [3]:
corpus = corpus.dropna(subset = ['text'])

In [4]:
corpus['docid'] = corpus['id'].str.split('\.').str[0:3].apply('_'.join)

In [5]:
corpus = corpus.groupby('docid')['text'].apply(','.join).reset_index()

In [6]:
tokens = corpus['text'].str.split(',').fillna('')

In [7]:
vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase = False)
vectorizer.fit(tokens)

C:\Users\Ben\anaconda3\envs\embed\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


CountVectorizer(lowercase=False,
                tokenizer=<function <lambda> at 0x00000188919113A8>)

In [8]:
vocab = vectorizer.vocabulary_

In [9]:
def get_matrices(sample_tokens):
    tfidf = TfidfVectorizer(tokenizer = lambda x: x, lowercase = False, vocabulary = vocab)
    tfidf_mtx = tfidf.fit_transform(sample_tokens)
    vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase = False, vocabulary = vocab)
    dtm = vectorizer.fit_transform(sample_tokens)
    return tfidf_mtx, dtm

In [10]:
seed(932487)
seeds = randint(1, 100000, 100)

In [11]:
def init_lda(i):
    seed(i)
    ncomp = choice(range(20, 310, 10))
    prior = choice(range(1, 50, 1)) / ncomp
    lda = LatentDirichletAllocation(n_components = ncomp,
                                   doc_topic_prior = prior,
                                   topic_word_prior = prior,
                                   n_jobs = -1,
                                   random_state = i)
    return lda    

In [12]:
def run_lda(i, tokens):
    sample_tokens = tokens.sample(n = len(tokens), replace = True, random_state = i)
    tfidf_mtx, dtm = get_matrices(sample_tokens)
    lda = init_lda(i)
    lda.fit(dtm)
    return lda.components_, tfidf_mtx

In [13]:
matrix_list = [run_lda(i, tokens) for i in seeds]

In [17]:
import numpy as np

In [31]:
matrix_list[0][1].shape

(21086, 44640)

In [32]:
matrix_list[0][0].shape

(80, 44640)

In [36]:
tfidf = matrix_list[0][1].todense()

In [39]:
tfidf[0].shape

(1, 44640)

In [45]:
matrix_list[0][0].shape

(80, 44640)

In [71]:
test = [np.multiply(matrix_list[0][0], tfidf[i]) for i in range(tfidf.shape[0])]

MemoryError: Unable to allocate 27.2 MiB for an array with shape (80, 44640) and data type float64

In [70]:
test = 

ValueError: operands could not be broadcast together with shapes (80,44640) (44640,21086) 

In [72]:
test.shape

(80, 44640)